<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
brings the initial data into the system for further processing by subsequent layers
### Hidden Layer:
a layer between the input and output layers where neurons take in a set of weighted inputs and produce an output through an activation function
### Output Layer:
the last layer of neurons that produces given outputs for the program
### Neuron:
a mathematical operation that takes its input, multiplies by its weight, and passes the sum through an activation function to other neurons
### Weight:
the parameter within a neural network that transforms input data within the network's hidden layers
### Activation Function:
defines the output of a neuron given an input / set of inputs
### Node Map:
a visual representation of the network's architecture, showing the input layer, hidden layer(s), and output layer, and the number of inputs and outputs in each layer
### Perceptron:
the simplest form of feed-forward neural network, a single-layer neural network -- can correctly classify linearly separable data

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

Set of inputs -> weights multiplied by each input -> bias added to each result -> passed to activation function -> result passed to next layer

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [4]:
##### Your Code Here #####
import numpy as np
def nand(x1, x2):
    x = np.array([1, x1, x2])
    w = np.array([1.5, -1, -1])
    y = np.dot(x, w)
    if y <= 0:
        return 0
    else:
        return 1

In [8]:
print("NAND Gate:")
for x1, x2 in zip(df['x1'], df['x2']):
    y = nand(x1, x2)
    print(f"({x1}, {x2}) -> {y}")

NAND Gate:
(0, 0) -> 1
(1, 0) -> 1
(0, 1) -> 1
(1, 1) -> 0


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [9]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [14]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

transformer = Normalizer().fit(diabetes[feats])
X = transformer.transform(diabetes[feats])

In [16]:
X.shape[1]

8

In [277]:
##### Update this Class #####
class Perceptron(object):
    
    def __init__(self, rate=0.01, niter=100, early_stopping=None):
        self.niter = niter
        self.rate = rate
        self.early_stopping = early_stopping

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        # Initialize weights
        self.weights = np.zeros(X.shape[1] + 1)
        
        self.errors = []
        
        min_error = 999999
        min_idx = 0

        for i in range(self.niter):
            sum_error = 0.0
            for row, target in zip(X, y):
                prediction = self.predict(row)
                error = target - prediction
                sum_error += error ** 2
                delta = self.rate * error
                self.weights[1:] += row * delta
                self.weights[0] += delta
            self.errors.append(sum_error)
            if i % 100 == 0:
                print(f'> epoch={i:d}, lrate={self.rate:.3f}, error={sum_error:.3f}')
            
            if self.early_stopping is not None:
                if i > (self.early_stopping - 1):
                    if sum_error < min_error:
                        min_error = sum_error
                        min_idx = i
                    if i >= self.early_stopping + min_idx:
                        if i % 100 != 0:
                            print(f'> epoch={i:d}, lrate={self.rate:.3f}, error={sum_error:.3f}')
                        print('-----EARLY STOPPING-----')
                        print(f'Epoch {min_idx} had the minimum error: {min_error:.3f}')
                        break
            
                
        return self.weights
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0)

In [278]:
pn = Perceptron(0.1, 10000, 50)
pn.fit(X, diabetes['Outcome'])

> epoch=0, lrate=0.100, error=326.000
> epoch=100, lrate=0.100, error=316.000
> epoch=114, lrate=0.100, error=309.000
-----EARLY STOPPING-----
Epoch 64 had the minimum error: 298.000


array([ 0.2       ,  0.84848942, -0.0446274 , -0.24038541,  0.00907112,
       -0.14018708,  0.00519631,  0.39123797, -0.24877392])

In [279]:
for i, row in enumerate(X):
    diabetes['Prediction'][i] = pn.predict(row)

/Users/weilu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [280]:
diabetes['Prediction'] = diabetes['Prediction'].apply(int)

In [281]:
diabetes['Prediction'].value_counts()

0    443
1    325
Name: Prediction, dtype: int64

In [282]:
diabetes['Outcome'].value_counts(normalize=True)

0    0.651042
1    0.348958
Name: Outcome, dtype: float64

In [283]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(diabetes['Outcome'], diabetes['Prediction'])
print(f'Accuracy: {accuracy:.6f}')

Accuracy: 0.610677


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?